In [ ]:
import os, streamlit as st, psycopg2, pandas as pd, plotly.express as px

# ── DB credentials ──
DB = dict(
    host     = st.secrets["DB_HOST"]     if "DB_HOST"     in st.secrets else "localhost",
    dbname   = st.secrets["DB_NAME"]     if "DB_NAME"     in st.secrets else "paintings_submission",
    user     = st.secrets["DB_USER"]     if "DB_USER"     in st.secrets else "postgres",
    password = st.secrets["DB_PASSWORD"] if "DB_PASSWORD" in st.secrets else "Asodit7878@95",
    port     = st.secrets.get("DB_PORT", 5432),
)

@st.cache_data(show_spinner=False)
def run_query(q, params=None):
    with psycopg2.connect(**DB) as conn:
        return pd.read_sql_query(q, conn, params=params)

st.title("🎨 Paintings Database Explorer")
page = st.sidebar.selectbox("Page",
        ["Artists", "Museums", "Works by Style", "Price-vs-Size"])

if page == "Artists":
    st.dataframe(run_query(
        "SELECT artist_id,last_name,first_name,nationality,style,birth,death "
        "FROM artist ORDER BY last_name"))
elif page == "Museums":
    st.dataframe(run_query(
        "SELECT m.museum_id,name,pc.city,pc.state,pc.country,url "
        "FROM museum m LEFT JOIN postalcode pc USING(postal)"
        "ORDER BY country,state,city").set_index("museum_id"))
elif page == "Works by Style":
    df = run_query(
        "SELECT style,COUNT(*) AS works FROM work WHERE style IS NOT NULL "
        "GROUP BY style ORDER BY works DESC LIMIT 20")
    st.plotly_chart(
        px.bar(df, x="style", y="works", title="Top 20 styles"), use_container_width=True)
else:
    df = run_query(
        "SELECT cs.width*cs.height AS area, ps.sale_price, w.style "
        "FROM product_size ps JOIN canvas_size cs USING(size_id) JOIN work w USING(work_id) "
        "WHERE ps.sale_price IS NOT NULL")
    st.plotly_chart(
        px.scatter(df, x="area", y="sale_price", color="style",
                   labels=dict(area="Canvas area", sale_price="Sale price")),
        use_container_width=True)
